# Restaurant Data Analysis — Colab Notebook

This Colab-ready notebook is prepared for the Cognifyz internship project. It includes data loading from Google Drive, data cleaning, EDA (Level 1, 2, 3 tasks), visualizations, and exporting results back to Google Drive.

**How to use:**
1. Upload `Dataset.csv` to your Google Drive folder `MyDrive/Cognifyz_Project/`.
2. Open this notebook in Google Colab (Upload the .ipynb or open from Drive).
3. Run cells in order. When prompted, authorize Google Drive access.

---

In [ ]:
# Mount Google Drive (Run this in Colab)
from google.colab import drive
drive.mount('/content/drive')

# Set path to project folder on your Drive
project_path = '/content/drive/MyDrive/Cognifyz_Project'
print('Project path set to:', project_path)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

pd.set_option('display.max_columns', 50)
plt.rcParams['figure.figsize'] = (8,5)


In [ ]:
# Load dataset from Drive
file_path = project_path + '/Dataset.csv'
try:
    df = pd.read_csv(file_path)
    print('Loaded dataset with shape:', df.shape)
except Exception as e:
    print('Error loading file. Make sure Dataset.csv is in', project_path)
    raise e


In [ ]:
# Quick inspection
print(df.head())
print('\nMissing values per column:')
print(df.isnull().sum())


## Data Cleaning
- Drop rows with missing `Cuisines` (if any).
- Convert yes/no columns to consistent format.
- Create helper columns where useful.

In [ ]:
# Data cleaning
initial_rows = df.shape[0]
df = df.dropna(subset=['Cuisines']).copy()
print(f'Dropped {initial_rows - df.shape[0]} rows with missing cuisines. Remaining rows: {df.shape[0]}')

# Standardize Yes/No
for col in ['Has Online delivery','Has Table booking','Is delivering now']:
    if col in df.columns:
        df[col] = df[col].str.strip().replace({'YES':'Yes','yes':'Yes','No':'No','NO':'No'})

# Ensure Price range is integer
if df['Price range'].dtype != int:
    df['Price range'] = df['Price range'].astype(int)

# Save a cleaned copy
cleaned_path = project_path + '/Cleaned_Dataset.csv'
df.to_csv(cleaned_path, index=False)
print('Cleaned dataset saved to', cleaned_path)


## Level 1 — Task 1: Top Cuisines
Count top cuisines and show percentages.

In [ ]:
# Top cuisines
cuisine_list = df['Cuisines'].str.split(', ').sum()
cuisine_counts = Counter(cuisine_list)
top3 = cuisine_counts.most_common(3)
print('Top 3 cuisines:')
for c, cnt in top3:
    print(f'{c}: {cnt} ({cnt/len(df)*100:.2f}%)')

# Create DataFrame
top_cuisine_df = pd.DataFrame(cuisine_counts.most_common(), columns=['Cuisine','Count'])
top_cuisine_df.head(10)


## Level 1 — Task 2: City Analysis
Restaurant count per city and average rating.

In [ ]:
# City counts and average rating
city_counts = df['City'].value_counts()
city_avg_rating = df.groupby('City')['Aggregate rating'].mean().sort_values(ascending=False)
print('Top 5 cities by restaurant count:')
print(city_counts.head())
print('\nTop 10 cities by average rating:')
print(city_avg_rating.head(10))


## Level 1 — Task 3: Price Range Distribution
Visualize price range distribution and percentages.

In [ ]:
# Price range distribution
price_counts = df['Price range'].value_counts().sort_index()
price_pct = (price_counts/len(df))*100
print(price_counts)
print('\nPercentage:')
print(price_pct)

plt.bar(price_counts.index, price_counts.values)
plt.xlabel('Price range')
plt.ylabel('Number of restaurants')
plt.title('Price Range Distribution')
plt.show()


## Level 1 — Task 4: Online Delivery
Compare restaurants with and without online delivery.

In [ ]:
# Online delivery stats
delivery_counts = df['Has Online delivery'].value_counts()
delivery_pct = (delivery_counts/len(df))*100
delivery_avg = df.groupby('Has Online delivery')['Aggregate rating'].mean()
print(delivery_counts)
print('\nPercentages:')
print(delivery_pct)
print('\nAverage rating by online delivery:')
print(delivery_avg)

plt.pie(delivery_counts.values, labels=delivery_counts.index, autopct='%1.1f%%')
plt.title('Online Delivery Availability')
plt.show()


## Level 2 — Task 1: Cuisine Combinations
Analyze most common cuisine pairs.

In [ ]:
# Cuisine combinations (pairs)
combo_list = []
for cuisines in df['Cuisines'].dropna():
    parts = cuisines.split(', ')
    if len(parts) > 1:
        combo_list.extend(combinations(parts, 2))
combo_counts = Counter(combo_list)
combo_top10 = combo_counts.most_common(10)
combo_df = pd.DataFrame([(f'{a} + {b}', c) for (a,b),c in combo_top10], columns=['Combination','Count'])
combo_df


## Level 2 — Task 2: Geographic Analysis
Average lat/long per city and mapping note.

In [ ]:
# Geographic analysis - mean coordinates per city
geo_df = df.groupby('City')[['Latitude','Longitude']].mean().reset_index()
geo_df.head()

# For mapping: use folium or export lat/long and use Google My Maps, Power BI, or Tableau for interactive plots.


## Level 2 — Task 3: Restaurant Chains
Identify restaurants appearing multiple times (possible chains).

In [ ]:
# Restaurant chains by name frequency
chain_counts = df['Restaurant Name'].value_counts()
chains = chain_counts[chain_counts > 1].reset_index()
chains.columns = ['Restaurant Name','Branch Count']
chains.head(30)


## Level 2 — Task 4: Reviews
Summary of rating text categories.

In [ ]:
# Rating text distribution
rating_text_counts = df['Rating text'].value_counts()
rating_text_counts


## Level 3 — Task 1: Votes Analysis
Max/min votes and correlation with rating.

In [ ]:
# Votes analysis
max_votes_idx = df['Votes'].idxmax()
min_votes_idx = df['Votes'].idxmin()
max_votes_restaurant = df.loc[max_votes_idx, ['Restaurant Name','Votes','Aggregate rating']]
min_votes_restaurant = df.loc[min_votes_idx, ['Restaurant Name','Votes','Aggregate rating']]
votes_corr = df['Votes'].corr(df['Aggregate rating'])
print('Max votes restaurant:')
print(max_votes_restaurant)
print('\nMin votes restaurant:')
print(min_votes_restaurant)
print(f"\nCorrelation between Votes and Rating: {votes_corr:.4f}")


## Level 3 — Task 2: Price vs Online Delivery & Table Booking
Analyze service availability by price range.

In [ ]:
# Price vs services
price_service = df.groupby('Price range')[['Has Online delivery','Has Table booking']].agg(lambda x: (x=='Yes').sum())
price_service['Total'] = df.groupby('Price range').size()
price_service['OnlineDelivery_pct'] = price_service['Has Online delivery'] / price_service['Total'] * 100
price_service['TableBooking_pct'] = price_service['Has Table booking'] / price_service['Total'] * 100
price_service


## Export results
Save key outputs back to your Google Drive folder for submission.

In [ ]:
# Save outputs to Drive
output_excel = project_path + '/Premkumar_DataAnalysis_Final.xlsx'
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    top_cuisine_df.to_excel(writer, sheet_name='Top Cuisines', index=False)
    city_counts.reset_index().to_excel(writer, sheet_name='City Counts', index=False)
    city_avg_rating.reset_index().to_excel(writer, sheet_name='City Avg Rating', index=False)
    price_counts.reset_index().to_excel(writer, sheet_name='Price Range', index=False)
    delivery_counts.reset_index().to_excel(writer, sheet_name='Online Delivery', index=False)
    combo_df.to_excel(writer, sheet_name='Cuisine Combos', index=False)
    geo_df.to_excel(writer, sheet_name='Geo Analysis', index=False)
    chains.to_excel(writer, sheet_name='Chains', index=False)
    rating_text_counts.reset_index().to_excel(writer, sheet_name='Rating Text', index=False)
    pd.DataFrame([{'MaxVotesRestaurant': max_votes_restaurant['Restaurant Name'], 'MaxVotes': int(max_votes_restaurant['Votes']), 'MinVotesRestaurant': min_votes_restaurant['Restaurant Name'], 'MinVotes': int(min_votes_restaurant['Votes']), 'VotesRatingCorr': votes_corr}]).to_excel(writer, sheet_name='Votes Summary', index=False)
    price_service.reset_index().to_excel(writer, sheet_name='Price vs Services', index=False)

print('Saved Excel to', output_excel)


## Final Notes
- After running, download the Excel and README, ZIP them with your notebook, and upload via the Cognifyz submission form.
- To share the Colab notebook: File -> Save a copy in Drive, then Share -> Anyone with the link (Viewer).